## Aplicación Práctica de los Modelos

###### Créditos: para la implementación de este código se ha hecho uso en parte de la inteligencia artificial ChatGPT 4.o y Gemini 2.5 Flash

In [ ]:
# Leer y preparar los datos, leer dfs

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

tweets16_dem = pd.read_csv('tweets16_dem.csv')
tweets16_rep = pd.read_csv('tweets16_rep.csv')

tweets20_dem = pd.read_csv('tweets20_dem.csv')
tweets20_rep = pd.read_csv('tweets20_rep.csv')

tweets24_dem = pd.read_csv('tweets24_dem.csv')
tweets24_rep = pd.read_csv('tweets24_rep.csv')

In [ ]:
# Comprobar tamaños y usuarios en común
print('tweets16_dem:', tweets16_dem.shape)
print('tweets16_rep:', tweets16_rep.shape)
print('tweets20_dem:', tweets20_dem.shape)
print('tweets20_rep:', tweets20_rep.shape)
print('tweets24_dem:', tweets24_dem.shape)
print('tweets24_rep:', tweets24_rep.shape)

tweets16_dem.head()

In [ ]:
# Comprobar que no haya ningún nan ni none en todo el df
print(tweets16_dem.isnull().sum())
print(tweets16_rep.isnull().sum())
print(tweets20_dem.isnull().sum())
print(tweets20_rep.isnull().sum())
print(tweets24_dem.isnull().sum())
print(tweets24_rep.isnull().sum())

print(tweets16_dem.isna().sum())
print(tweets16_rep.isna().sum())
print(tweets20_dem.isna().sum())
print(tweets20_rep.isna().sum())
print(tweets24_dem.isna().sum())
print(tweets24_rep.isna().sum())

In [ ]:
# Comprobar usuarios en común de la columna "User"
print('tweets16_dem:', len(tweets16_dem['User'].unique()))
print('tweets16_rep:', len(tweets16_rep['User'].unique()))
print('tweets20_dem:', len(tweets20_dem['User'].unique()))
print('tweets20_rep:', len(tweets20_rep['User'].unique()))
print('tweets24_dem:', len(tweets24_dem['User'].unique()))
print('tweets24_rep:', len(tweets24_rep['User'].unique()))

# Hacemos las intersecciones de los usuarios
inter_16 = set(tweets16_dem['User'].unique()).intersection(set(tweets16_rep['User'].unique()))
inter_20 = set(tweets20_dem['User'].unique()).intersection(set(tweets20_rep['User'].unique()))
inter_24 = set(tweets24_dem['User'].unique()).intersection(set(tweets24_rep['User'].unique()))
print('inter_16:', len(inter_16))
print('inter_20:', len(inter_20))
print('inter_24:', len(inter_24))

# Comprobar si hay usuarios en común entre los años haciendo las intersecciones entre inter_16, inter_20 e inter_24
inter_16_20 = inter_16.intersection(inter_20)
inter_16_24 = inter_16.intersection(inter_24)
inter_20_24 = inter_20.intersection(inter_24)
print('inter_16_20:', len(inter_16_20))
print('inter_16_24:', len(inter_16_24))
print('inter_20_24:', len(inter_20_24))


In [ ]:
# Sacar número de usuarios totales por año

print('users 16: ' + str(len(tweets16_dem['User'].unique()) + len(tweets16_rep['User'].unique()) - len(inter_16)))
print('users 20: ' + str(len(tweets20_dem['User'].unique()) + len(tweets20_rep['User'].unique()) - len(inter_20)))
print('users 24: ' + str(len(tweets24_dem['User'].unique()) + len(tweets24_rep['User'].unique()) - len(inter_24)))

In [ ]:
def build_vote_intention(tweets_rep, tweets_dem):
    # Valoración y número de tweets para los republicanos en 2016
    rep_summary = tweets_rep.groupby('User').agg(
        u_R=('Label', 'mean'),
        m_R=('Label', 'count')
    )

    # Valoración y número de tweets para los demócratas en 2016
    dem_summary = tweets_dem.groupby('User').agg(
        u_D=('Label', 'mean'),
        m_D=('Label', 'count')
    )

    # Unimos ambos resúmenes teniendo en cuenta todos los usuarios (outer join)
    vote_df = rep_summary.join(dem_summary, how='outer').fillna(0)

    # Calculamos vote_intention usando la fórmula:
    # vote_intention = (u_R*m_R - u_D*m_D) / (m_R + m_D)
    # Para el caso en que (m_R + m_D)==0, definimos vote_intention = 0
    vote_df['vote_intention'] = vote_df.apply(
        lambda row: (row['u_R']*row['m_R'] - row['u_D']*row['m_D']) / (row['m_R'] + row['m_D'])
                    if (row['m_R'] + row['m_D']) != 0 else 0,
        axis=1
    )

    return vote_df


In [ ]:
V_df_16 = build_vote_intention(tweets16_rep, tweets16_dem)
V_df_20 = build_vote_intention(tweets20_rep, tweets20_dem)
V_df_24 = build_vote_intention(tweets24_rep, tweets24_dem)

print('V_16:', V_df_16)
print('V_20:', V_df_20)
print('V_24:', V_df_24)

In [ ]:
# el data_table: Una estructura de tabla con columnas 'nombre de usuario', 'tweet', 'etiqueta'. Las etiquetas son 'dem', 'neu', o 'rep'.
# Cogemos las tablas originales y en la de rep si Label == 1 la cambiamos a 'rep' y si es -1 a 'dem' y si es 0 a 'neu' y en la tabla dem al revés

def build_data_table(tweets_rep, tweets_dem):
    # Cambiamos las etiquetas de los tweets republicanos
    tweets_rep['Label'] = tweets_rep['Label'].replace({1: 'rep', -1: 'dem', 0: 'neu'})
    # Cambiamos las etiquetas de los tweets demócratas
    tweets_dem['Label'] = tweets_dem['Label'].replace({-1: 'rep', 1: 'dem', 0: 'neu'})

    # Concatenamos ambos dataframes
    data_table = pd.concat([tweets_rep, tweets_dem], ignore_index=True)

    return data_table

In [ ]:
data_table_16 = build_data_table(tweets16_rep, tweets16_dem)
data_table_20 = build_data_table(tweets20_rep, tweets20_dem)
data_table_24 = build_data_table(tweets24_rep, tweets24_dem)
print('data_table_16:', data_table_16)
print('data_table_20:', data_table_20)
print('data_table_24:', data_table_24)

In [ ]:
# Preparamos los parámetros de los índices

# vote_intention
V_16 = V_df_16['vote_intention'].values
V_20 = V_df_20['vote_intention'].values
V_24 = V_df_24['vote_intention'].values

# vote_intention_[0,1] (la traslación de (vote_intention + 1)/2 )
V_16_traslacion = (V_df_16['vote_intention'] + 1) / 2
V_20_traslacion = (V_df_20['vote_intention'] + 1) / 2
V_24_traslacion = (V_df_24['vote_intention'] + 1) / 2

In [ ]:
# Mostramos la distribución graficamente de vote_intention
plt.figure(figsize=(10, 5))
plt.hist(V_16, bins=20, alpha=0.5, label='2016')
plt.xlabel('Vote Intention')
plt.ylabel('Frequency')
plt.title('Vote Intention Distribution')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(V_20, bins=20, alpha=0.5, label='2020')
plt.xlabel('Vote Intention')
plt.ylabel('Frequency')
plt.title('Vote Intention Distribution')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(V_24, bins=20, alpha=0.5, label='2024')
plt.xlabel('Vote Intention')
plt.ylabel('Frequency')
plt.title('Vote Intention Distribution')
plt.legend()
plt.show()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('data_table_16:')
data_table_16

In [ ]:
# muestra de las data_table usuarios que tengan algunos tweets con etiquetas diferentes

# Filtrar los usuarios que tienen tweets con al menos dos etiquetas diferentes
users_with_both = (
    data_table_16.groupby('User')['Label']
    .apply(lambda etiquetas: len(set(etiquetas)) >= 2)
)
selected_users = users_with_both[users_with_both].index.tolist()
subset_data_table = data_table_16[data_table_16['User'].isin(selected_users)]

print("Usuarios con tweets de al menos dos etiquetas diferentes:")
print(len(subset_data_table['User'].unique()))

In [ ]:
users_with_both = (
    data_table_20.groupby('User')['Label']
    .apply(lambda etiquetas: len(set(etiquetas)) >= 2)
)
selected_users = users_with_both[users_with_both].index.tolist()
subset_data_table = data_table_20[data_table_20['User'].isin(selected_users)]

print("Usuarios con tweets de al menos dos etiquetas diferentes:")
print(len(subset_data_table['User'].unique()))

In [ ]:
users_with_both = (
    data_table_24.groupby('User')['Label']
    .apply(lambda etiquetas: len(set(etiquetas)) >= 2)
)
selected_users = users_with_both[users_with_both].index.tolist()
subset_data_table = data_table_24[data_table_24['User'].isin(selected_users)]

print("Usuarios con tweets de al menos dos etiquetas diferentes:")
print(len(subset_data_table['User'].unique()))

#### Índice de Polarización Gravitatoria

In [ ]:
import numpy as np

def calcular_IPG(V):
    """
    Calcula el Índice de Polarización Gravitatoria (IPG) para un vector
    discreto de opiniones V.

    Args:
        V: Un vector (lista o array de numpy) de valores numéricos
           discretos, representando opiniones en el rango [-1, 1].

    Returns:
        El valor del IPG calculado, o 0 si el vector de entrada está vacío.
    """
    N = len(V)
    if N == 0:
        return 0

    # --- Paso 1: Calcular probabilidades de masa ---
    # Contar el número de usuarios con opiniones < 0, == 0, > 0
    count_neg = sum(1 for v in V if v < 0)
    count_zero = sum(1 for v in V if v == 0)
    count_pos = sum(1 for v in V if v > 0)

    # Calcular las probabilidades de masa P(X < 0), P(X = 0), P(X > 0)
    # Las probabilidades son proporciones sobre el total de usuarios.
    prob_neg = count_neg / N
    prob_zero = count_zero / N
    prob_pos = count_pos / N
    prob_not_zero = prob_neg + prob_pos # O 1.0 - prob_zero

    # --- Paso 2: Calcular A+ y A- ---
    A_plus = 0
    A_minus = 0
    if prob_not_zero > 0: # Evitar división por cero si todos los puntos son 0
         A_plus = prob_pos * (1 + prob_zero / prob_not_zero)
         A_minus = prob_neg * (1 + prob_zero / prob_not_zero)

    # --- Paso 3: Calcular la diferencia normalizada en el tamaño de las poblaciones ---
    delta_A = A_plus - A_minus

    # --- Paso 4: Calcular los centros de gravedad gc+ y gc- ---
    # La integral de p(x)*x dx para una variable discreta es sumatorio(x * p(x))
    # sum_{x en V, x>0} x * p(x) = sum_{x perteneciente a V, x>0} x * (# de x en V / N)
    # Esto es igual a (sumatorio de todos los valores positivos en V incluyendo repeticiones) / N
    sum_pos_values = sum(v for v in V if v > 0)
    sum_neg_values = sum(v for v in V if v < 0)

    numerator_gc_pos = sum_pos_values / N
    numerator_gc_minus = sum_neg_values / N

    gc_plus = 0
    if A_plus > 0:
        gc_plus = numerator_gc_pos / A_plus

    gc_minus = 0
    if A_minus > 0:
        gc_minus = numerator_gc_minus / A_minus

    # --- Paso 5: Calcular la distancia normalizada entre los centros de gravedad ---
    d = abs(gc_plus - gc_minus) / 2.0 # Dividido por el rango total [-1, 1] que es 2

    # --- Paso 6: Calcular el Índice de Polarización Gravitatoria (IPG) ---
    mu = (1 - abs(delta_A)) * d

    return mu


In [ ]:
# Ejemplo de uso:
V_ejemplo = [-1, -1, -0.5, 0, 0.5, 0.5, 0.5, 1, 1]
ipg_calculado = calcular_IPG(V_ejemplo)
print(f"El vector V es: {V_ejemplo}")
print(f"El IPG calculado es: {ipg_calculado}")

V_ejemplo_neutro = [0, 0, 0, 0, 0]
ipg_neutro = calcular_IPG(V_ejemplo_neutro)
print(f"\nEl vector V es: {V_ejemplo_neutro}")
print(f"El IPG calculado es: {ipg_neutro}")

V_ejemplo_polarizado = [-1, -1, -1, 1, 1, 1]
ipg_polarizado = calcular_IPG(V_ejemplo_polarizado)
print(f"\nEl vector V es: {V_ejemplo_polarizado}")
print(f"El IPG calculado es: {ipg_polarizado}")

V_ejemplo_sesgado = [-1, -1, -1, -1, -1, 1]
ipg_sesgado = calcular_IPG(V_ejemplo_sesgado)
print(f"\nEl vector V es: {V_ejemplo_sesgado}")
print(f"El IPG calculado es: {ipg_sesgado}")

In [ ]:
# Aplicación a nuestras muestras

IPG_16 = calcular_IPG(V_16)
IPG_20 = calcular_IPG(V_20)
IPG_24 = calcular_IPG(V_24)

print(IPG_16)
print(IPG_20)
print(IPG_24)

#### Índice de Foster y Wolfson

In [ ]:
import numpy as np

def gini(x):
    # Asegúrate de que no haya valores negativos
    x = np.array(x)
    if np.amin(x) < 0:
        raise ValueError("Los valores no pueden ser negativos")

    # Si todo es cero, el Gini no está definido (retornamos 0 por convención)
    if np.all(x == 0):
        return 0.0

    # Ordenamos
    x_sorted = np.sort(x)
    n = len(x)
    index = np.arange(1, n + 1)

    # Fórmula del índice de Gini
    return (2 * np.sum(index * x_sorted)) / (n * np.sum(x_sorted)) - (n + 1) / n


In [ ]:
import numpy as np
import math

def calcular_FW(V):
    """
    Calcula el índice de Foster-Wolfson P para un vector discreto V,
    asumiendo que V contiene valores en el intervalo [0, 1]

    Args:
        V: Un vector (lista o array de numpy) de valores numéricos
           en el rango [0, 1]. 

    Returns:
        El valor del índice P calculado. Retorna 0.0 si el vector
        está vacío o tiene un solo elemento.
    """
    V = np.array(V, dtype=float) # Asegurar tipo float
    V = V[~np.isnan(V)] # Limpiar NaNs

    N = len(V)

    # Casos degenerados: vector vacío o un solo elemento
    # En estos casos, la polarización es 0 y las fórmulas no aplican directamente.
    if N == 0 or N == 1:
        return 0.0

    # 1. Calcular mu (media aritmética) y m (mediana) de toda la población
    mu = np.mean(V)
    m = np.median(V)

    if mu == 0 or m == 0:
        return 0.0

    # 2. Calcular G (Índice de Gini)
    # Usamos scipy.stats.gini que espera datos no negativos, lo cual se cumple con [0, 1].
    G = gini(V)

    # 3. Calcular T = (mu+ - mu-) / mu
    # Dividir V en dos grupos de tamaño N//2 y N - N//2 para calcular mu+ y mu-.
    # Se sigue la regla de distribuir los valores iguales a la mediana para igualar tamaños.

    V_sorted = np.sort(V)

    # Número de elementos estrictamente por debajo/encima de la mediana *del conjunto completo*
    count_lt_m = np.sum(V < m)
    count_eq_m = np.sum(V == m)

    # Determinar cuántos valores iguales a la mediana deben ir al grupo "inferior"
    # para que este grupo alcance el tamaño N//2.
    target_lower_size = N // 2
    num_median_to_lower = target_lower_size - count_lt_m

    # Asegurarse de que num_median_to_lower esté dentro del rango [0, count_eq_m]
    # Esto debería cumplirse bajo la asunción de que m es la mediana real y N>=2,
    # pero max/min añaden robustez.
    num_median_to_lower = max(0, num_median_to_lower)
    num_median_to_lower = min(count_eq_m, num_median_to_lower)

    # El número de valores iguales a la mediana que van al grupo "superior"
    num_median_to_upper = count_eq_m - num_median_to_lower

    # Construir los dos sub-vectores para el cálculo de T
    # Grupo inferior: todos los < m y los num_median_to_lower valores iguales a m
    V_lower_for_T = np.concatenate((V_sorted[V_sorted < m], np.full(num_median_to_lower, m)))
    # Grupo superior: num_median_to_upper valores iguales a m y todos los > m
    V_upper_for_T = np.concatenate((np.full(num_median_to_upper, m), V_sorted[V_sorted > m]))

    # Calcular mu- y mu+ (medias de los sub-vectores)
    mu_minus = np.mean(V_lower_for_T)
    mu_plus = np.mean(V_upper_for_T)

    # Calcular T = (mu+ - mu-) / mu
    T = (mu_plus - mu_minus) / mu

    # 4. Calcular P = 2 * (mu / m) * (T - G)
    P = 2.0 * (mu / m) * (T - G)

    return P

In [ ]:
# --- Ejemplos de uso ---

# Ejemplo 1: Distribución bimodal simple en [0, 1]
V_ejemplo_bimodal_0_1 = np.array([0.1, 0.1, 0.2, 0.8, 0.9, 0.9])
print("--- Ejemplo 1 (Datos en [0, 1], Bimodal) ---")
print(f"Vector V: {V_ejemplo_bimodal_0_1}")
ipg_fw_0_1_1 = calcular_FW(V_ejemplo_bimodal_0_1)
print(f"Foster-Wolfson P: {ipg_fw_0_1_1}")
print(f"Media (mu): {np.mean(V_ejemplo_bimodal_0_1)}, Mediana (m): {np.median(V_ejemplo_bimodal_0_1)}")

# Ejemplo 2: Datos sesgados en [0, 1] con punto en la mediana
V_ejemplo_sesgado_0_1 = np.array([0.1, 0.2, 0.3, 0.5, 0.5, 0.8, 0.9, 0.9])
print("\n--- Ejemplo 2 (Datos en [0, 1], Sesgado con punto en mediana) ---")
print(f"Vector V: {V_ejemplo_sesgado_0_1}")
ipg_fw_0_1_2 = calcular_FW(V_ejemplo_sesgado_0_1)
print(f"Foster-Wolfson P: {ipg_fw_0_1_2}")
print(f"Media (mu): {np.mean(V_ejemplo_sesgado_0_1)}, Mediana (m): {np.median(V_ejemplo_sesgado_0_1)}")

# Ejemplo 3: Todos los puntos son iguales (pero no 0)
V_ejemplo_iguales_0_1 = np.array([0.7, 0.7, 0.7, 0.7])
print("\n--- Ejemplo 3 (Datos en [0, 1], Iguales y no cero) ---")
print(f"Vector V: {V_ejemplo_iguales_0_1}")
ipg_fw_0_1_3 = calcular_FW(V_ejemplo_iguales_0_1)
print(f"Foster-Wolfson P: {ipg_fw_0_1_3}") # Debería ser 0.0
print(f"Media (mu): {np.mean(V_ejemplo_iguales_0_1)}, Mediana (m): {np.median(V_ejemplo_iguales_0_1)}")

In [ ]:
# Aplicación a nuestras muestras

FW_16 = calcular_FW(V_16_traslacion)
FW_20 = calcular_FW(V_20_traslacion)
FW_24 = calcular_FW(V_24_traslacion)

print(FW_16)
print(FW_20)
print(FW_24)

#### Índice de Esteban y Ray

In [ ]:
import numpy as np
import math

def calcular_ER(V, alpha):
    """
    Calcula el índice de polarización de Esteban-Ray para un vector V en [-1, 1].

    Args:
        V: Un vector de opiniones en el rango [-1, 1].
        alpha: Hiperparámetro de la fórmula, debe estar en (0, 1.6].

    Returns:
        El valor del índice de polarización de Esteban-Ray calculado.
        Retorna 0.0 en casos degenerados (vector vacío, una sola opinión o son todos cero).

    Raises:
        ValueError: Si el valor de alpha está fuera del rango (0, 1.6].
    """
    # 1. Validar alpha
    if not (0 < alpha <= 1.6):
        raise ValueError("El hiperparámetro alpha debe estar en el rango (0, 1.6]")

    V = np.array(V, dtype=float) # Asegurar tipo float
    V = V[~np.isnan(V)] # Limpiar NaNs

    N = len(V)

    # Caso degenerado: vector vacío o un solo elemento
    if N == 0 or N == 1:
        return 0.0

    # 2. Separar opiniones en < 0, == 0, > 0 y obtener sus recuentos
    V_neg = V[V < 0]
    V_zero = V[V == 0] # Aunque no usamos el array V_zero directamente en sumas, necesitamos su count
    V_pos = V[V > 0]

    count_neg = len(V_neg)
    count_zero = len(V_zero)
    count_pos = len(V_pos)

    count_not_zero = count_neg + count_pos

    # 3. Casos degenerados donde no se pueden formar dos grupos opuestos significativos
    # Esto ocurre si no hay opiniones no-cero (todos son 0)
    # O si todas las opiniones no-cero tienen el mismo signo (count_neg == 0 O count_pos == 0)
    if count_not_zero == 0 or count_neg == 0 or count_pos == 0:
        # En estos casos, no hay dos polos opuestos significativos para distribuir ceros.
        # La polarización, según esta definición de grupos, es 0.
        return 0.0

    # 4. Distribuir las opiniones cero proporcionalmente a la división de los no-cero
    # Proporción de opiniones no-cero que son negativas
    prop_neg_among_not_zero = count_neg / count_not_zero
    # Proporción de opiniones no-cero que son positivas
    # prop_pos_among_not_zero = count_pos / count_not_zero # Esto es 1.0 - prop_neg_among_not_zero

    # Número (posiblemente fraccionario) de opiniones cero asignadas a cada grupo
    # Esta es la implementación de "repartiendo proporcionalmente" para los ceros
    zeros_to_democrats = count_zero * prop_neg_among_not_zero
    zeros_to_republicans = count_zero - zeros_to_democrats # Es equivalente a count_zero * (count_pos / count_not_zero)

    # 5. Definir los dos grupos finales para la fórmula de ER y calcular sus propiedades
    final_count_democrats = count_neg + zeros_to_democrats
    final_count_republicans = count_pos + zeros_to_republicans

    # Suma de valores en cada grupo. Las opiniones originales negativas/positivas contribuyen con su valor.
    # Las opiniones cero contribuyen con 0 a la suma, independientemente de cuántas vayan a cada grupo.
    sum_democrats_values = np.sum(V_neg)
    sum_republicans_values = np.sum(V_pos)

    # 6. Calcular proporciones (pi) y medias (x, y) para la fórmula de Esteban-Ray
    pi = final_count_democrats / N
    one_minus_pi = final_count_republicans / N # Es 1.0 - pi, calculado para simetría

    # Calcular medias x e y. Dividir suma de valores por el tamaño final (conceptual) del grupo.
    x = sum_democrats_values / final_count_democrats
    y = sum_republicans_values / final_count_republicans

    # 7. Calcular el índice de polarización P (Esteban-Ray fórmula)
    # pi^(1+alpha)*(1-pi) + (1-pi)^(1+alpha)*pi
    # pi y 1-pi están estrictamente entre 0 y 1 en este punto, por lo que las potencias son válidas.
    term_bracket = pi**(1.0 + alpha) * (1.0 - pi) + (1.0 - pi)**(1.0 + alpha) * pi

    # La diferencia entre las medias de los grupos.
    # x será la media de valores <= 0 (negativos y/o ceros).
    # y será la media de valores >= 0 (positivos y/o ceros).
    difference_in_means = y - x

    # Fórmula final del Índice de Polarización P
    P = term_bracket * difference_in_means

    return P


In [ ]:
# --- Ejemplos de uso ---

# Ejemplo 1: Bipolarización fuerte con algunos neutros
V_ejemplo_1 = np.array([-1, -1, -0.5, 0, 0, 0.5, 1, 1])
alpha_ejemplo = 1.0
print("--- Ejemplo 1 (Esteban-Ray con distribución proporcional de ceros) ---")
print(f"Vector V: {V_ejemplo_1}")
print(f"Alpha: {alpha_ejemplo}")
er_prop_zeros_1 = calcular_ER(V_ejemplo_1, alpha_ejemplo)
print(f"Esteban-Ray P: {er_prop_zeros_1}")


# Ejemplo 2: Bipolarización fuerte sin neutros (Igual que antes)
V_ejemplo_2 = np.array([-1, -1, -1, 1, 1, 1])
alpha_ejemplo = 0.5
print(f"\n--- Ejemplo 2 (Solo opiniones negativas y positivas) ---")
print(f"Vector V: {V_ejemplo_2}")
print(f"Alpha: {alpha_ejemplo}")
er_prop_zeros_2 = calcular_ER(V_ejemplo_2, alpha_ejemplo)
print(f"Esteban-Ray P: {er_prop_zeros_2}")

# Ejemplo 3: Con ceros y grupos no vacíos (El mismo que el 1, pero ahora lo probamos de nuevo)
V_ejemplo_5 = np.array([-1, -0.5, 0, 0, 0.5, 1])
alpha_ejemplo = 1.0
print(f"\n--- Ejemplo 5 (Con ceros, distribución proporcional) ---")
print(f"Vector V: {V_ejemplo_5}")
print(f"Alpha: {alpha_ejemplo}")
er_prop_zeros_5 = calcular_ER(V_ejemplo_5, alpha_ejemplo)
print(f"Esteban-Ray P: {er_prop_zeros_5}")


In [ ]:
# Aplicación a nuestras muestras

ER_16 = calcular_ER(V_16, 1.5)
ER_20 = calcular_ER(V_20, 1.5)
ER_24 = calcular_ER(V_24, 1.5)

print(ER_16)
print(ER_20)
print(ER_24)

#### Índice Beta de Polarización

In [ ]:
import pandas as pd
import numpy as np
import math

def calcular_entropia_usuario(data_table):
    """
    Calcula la firmeza de opinión (entropía normalizada h_i) para cada usuario
    basándose en la distribución de etiquetas ('dem', 'neu', 'rep') en sus tweets.

    Args:
        data_table: Una estructura de tabla con columnas
                    'User', 'tweet', 'Label'. Las etiquetas
                    son 'dem', 'neu', o 'rep'.

    Returns:
        Un diccionario {user_id: h_i}, donde h_i es la entropía normalizada [0, 1].
        Retorna 0.0 para usuarios sin tweets o con todos los tweets de la misma etiqueta.

    Raises:
        ValueError: Si la tabla de datos no contiene las columnas requeridas.
    """
    # Asegurar que la entrada es un DataFrame
    if not isinstance(data_table, pd.DataFrame):
        try:
            data_table = pd.DataFrame(data_table)
        except Exception as e:
            raise ValueError(f"No se pudo convertir data_table a DataFrame: {e}")

    if data_table.empty:
        return {}
    
    # Asegurar que las columnas necesarias existen
    required_cols = ['User', 'Label']
    if not all(col in data_table.columns for col in required_cols):
        missing = [col for col in required_cols if col not in data_table.columns]
        raise ValueError(f"data_table debe contener las columnas: {required_cols}. Faltan: {missing}")
    
    # Definir todas las etiquetas posibles
    all_labels = ['dem', 'neu', 'rep']

    # Agrupar por usuario y contar etiquetas
    # .unstack() convierte las etiquetas en columnas, fill_value=0 llena con ceros donde un usuario no tiene una etiqueta
    user_label_counts = data_table.groupby('User')['Label'].value_counts().unstack(fill_value=0)
    
    # Asegurar que todas las columnas de etiquetas ('dem', 'neu', 'rep') existan, incluso si ningún usuario las tiene
    for label in all_labels:
        if label not in user_label_counts.columns:
            user_label_counts[label] = 0

    # Reordenar columnas para que siempre estén en el orden 'dem', 'neu', 'rep'
    user_label_counts = user_label_counts[all_labels]

    # Calcular el total de tweets por usuario
    user_total_tweets = user_label_counts.sum(axis=1)
    
    # Calcular las proporciones p_ik. Evitar división por cero para usuarios sin tweets (aunque value_counts lo manejaría)
    # Reemplazamos 0 por NaN en el total para que la división dé NaN, luego fillna(0) para que la entropía sea 0.
    user_total_tweets_safe = user_total_tweets.replace(0, np.nan)
    user_label_proportions = user_label_counts.div(user_total_tweets_safe, axis=0).fillna(0.0) # .fillna(0.0) si el total es 0
    
    # Asegúrate de que user_label_proportions es puramente numérico
    ulp_values = user_label_proportions.astype(float).values 

    log_proportions = np.zeros_like(ulp_values)
    mask = ulp_values > 0
    log_proportions[mask] = np.log(ulp_values[mask])

    p_log_p = ulp_values * log_proportions 
    # p_log_p ya no debería tener NaNs de 0*log(0), 
    # pero np.nan_to_num no haría daño si los datos originales tuvieran NaNs.
    p_log_p = np.nan_to_num(p_log_p, nan=0.0) 

    # Sumar p_ik * log(p_ik) sobre las etiquetas para cada usuario
    sum_p_log_p_per_user = p_log_p.sum(axis=1)

    # Calcular entropía h_i = - (1 / log(3)) * sum(p_ik * log(p_ik)).
    log3 = np.log(3)
    
    h_i = -sum_p_log_p_per_user / log3 # h_i es un array
    
    # Obtener los nombres de usuario del índice del DataFrame de proporciones
    user_ids = user_label_proportions.index
    
    # Crear una Serie de Pandas para facilitar la conversión a diccionario
    h_i_series = pd.Series(h_i, index=user_ids)

    # El resultado como diccionario {user_id: h_i}
    return h_i_series.to_dict()


In [ ]:
def calcular_IB(V, beta, entropies_h_i, user_ids_for_V):
    """
    Calcula el Índice Beta de polarización según las fórmulas proporcionadas,
    incluyendo la distribución proporcional de usuarios cero y el uso de entropías.

    Args:
        V: Array de opiniones (vote_intention) en [-1, 1], alineado con user_ids_for_V.
        beta: Parámetro de peso de cohesión [0, 1].
        entropies_h_i: Diccionario {user_id: h_i} de entropías normalizadas por usuario,
                       calculadas por calcular_entropia_usuario.
        user_ids_for_V: Array de IDs de usuario, alineado con V (user_ids_for_V[k] es
                        el ID del usuario cuya opinión es V[k]).

    Returns:
        El valor del Índice Beta de Polarización [0, 1]. Retorna 0.0 en casos degenerados.

    Raises:
        ValueError: Si beta está fuera del rango [0, 1].
        ValueError: Si V y user_ids_for_V no tienen el mismo tamaño.
        ValueError: Si falta la entropía para algún usuario que aparece en V.
    """
    # 1. Validar beta
    if not (0 <= beta <= 1):
        raise ValueError("El parámetro beta debe estar en el rango [0, 1]")

    # Asegurar que las entradas sean arrays de numpy y tengan el mismo tamaño
    V = np.array(V, dtype=float)
    user_ids_for_V = np.array(user_ids_for_V)

    if len(V) != len(user_ids_for_V):
        raise ValueError("Los arrays V y user_ids_for_V deben tener el mismo tamaño.")

    # Limpiar NaNs de V y los user_ids correspondientes (manteniendo alineación)
    valid_mask = ~np.isnan(V)
    V = V[valid_mask]
    user_ids_for_V = user_ids_for_V[valid_mask]

    N = len(V) # Número total de usuarios válidos con opinión

    # Casos degenerados: vector vacío o un solo usuario
    if N == 0 or N == 1:
        return 0.0

    # 2. Separar usuarios por opinión: < 0, == 0, > 0
    neg_mask = V < 0
    zero_mask = V == 0
    pos_mask = V > 0

    users_neg = user_ids_for_V[neg_mask]
    users_zero = user_ids_for_V[zero_mask]
    users_pos = user_ids_for_V[pos_mask]

    count_neg = len(users_neg)
    count_zero = len(users_zero)
    count_pos = len(users_pos)

    count_not_zero = count_neg + count_pos

    # 3. Casos degenerados donde no se pueden formar dos polos opuestos significativos
    # Si no hay usuarios no-cero, o si todos los usuarios no-cero están en el mismo polo.
    if count_not_zero == 0 or count_neg == 0 or count_pos == 0:
        # No hay dos polos opuestos (D y R estrictos) entre los no-cero para distribuir los ceros.
        # La polarización bipartidista es 0.
        return 0.0

    # 4. Distribuir los usuarios cero proporcionalmente entre los polos D y R.
    # La proporción se basa en el tamaño relativo de los polos estrictos (no-cero).
    prop_neg_among_not_zero = count_neg / count_not_zero

    # Calcular cuántos usuarios cero van a cada polo. Redondeamos para obtener un número entero.
    # La suma de los redondeados debe ser igual a count_zero.
    num_zero_to_democrats = round(count_zero * prop_neg_among_not_zero)
    num_zero_to_republicans = count_zero - num_zero_to_democrats # El resto para asegurar que suman count_zero

    # 5. Definir los sets finales de usuarios para cada polo (D y R)
    # Seleccionamos usuarios cero para D y R de forma determinística (ej. por orden en el array original users_zero).
    users_zero_to_D = users_zero[:num_zero_to_democrats]
    users_zero_to_R = users_zero[num_zero_to_democrats:] # Los restantes usuarios cero

    # Los polos finales D y R son la unión de los usuarios estrictos y los cero asignados.
    D_users = np.concatenate((users_neg, users_zero_to_D))
    R_users = np.concatenate((users_pos, users_zero_to_R))

    # abs_D = |D|, abs_R = |R|
    abs_D = len(D_users)
    abs_R = len(R_users)

    if abs_D == 0 or abs_R == 0:
         return 0.0

    # 6. Calcular m_D y m_R (mediana de la intensidad absoluta dentro de cada polo)
    # Obtener los valores de opinión V correspondientes a los usuarios en cada polo final
    # Nota: V y user_ids_for_V están alineados. Usamos np.isin para filtrar V basado en los IDs de los polos.
    V_D_values = V[np.isin(user_ids_for_V, D_users)]
    V_R_values = V[np.isin(user_ids_for_V, R_users)]

    # Calcular la mediana de los valores absolutos dentro de cada polo
    # Los valores absolutos de los usuarios cero son |0| = 0.
    m_D = np.median(np.abs(V_D_values))
    m_R = np.median(np.abs(V_R_values))

    # 7. Calcular H_D y H_R (cohesión interna / entropía media por polo)
    # Sumar las entropías h_i para todos los usuarios en cada polo final
    sum_h_D = 0.0
    for user_id in D_users:
        if user_id not in entropies_h_i:
             # Todos los usuarios que aparecen en V deben tener una entropía calculada
             raise ValueError(f"Falta la entropía (h_i) para el usuario '{user_id}' en el diccionario provided.")
        sum_h_D += entropies_h_i[user_id]

    sum_h_R = 0.0
    for user_id in R_users:
        if user_id not in entropies_h_i:
             raise ValueError(f"Falta la entropía (h_i) para el usuario '{user_id}' en el diccionario provided.")
        sum_h_R += entropies_h_i[user_id]

    # Calcular las entropías medias (H_D, H_R)
    # abs_D y abs_R son > 0 aquí.
    H_D = sum_h_D / abs_D
    H_R = sum_h_R / abs_R

    # 8. Calcular p (fracción del bloque minoritario)
    p = min(abs_D, abs_R) / N

    # 9. Calcular el Índice Beta final
    # |m_D| y |m_R| ya son >= 0 por np.abs()
    median_distance_term = m_D + m_R

    # Componente de cohesión [0, 1 - beta*1] => [0, 1] ya que beta está en [0, 1] y (H_D+H_R)/2 está en [0, 1]
    cohesion_term = 1 - beta * (H_D + H_R) / 2

    # Índice Beta
    indice_beta = median_distance_term * cohesion_term * p

    # El índice se define en [0, 1].
    # max(|m_D|+|m_R|) = 2 (ej. todos -1 y todos 1, distribuidos los 0s).
    # max(cohesion_term) = 1 (cuando beta=0 o H_D+H_R=0).
    # max(p) = 0.5 (cuando |D|=|R|=N/2).
    # Max Index = 2 * 1 * 0.5 = 1.0. El rango [0, 1] es correcto.

    return indice_beta


In [ ]:
# --- Ejemplos de uso ---

# 1. Crear datos de ejemplo
# Tabla de datos para calcular entropía
data_tweets_ejemplo = pd.DataFrame({
    'User': ['user1', 'user1', 'user1', 'user2', 'user2', 'user3', 'user3', 'user3', 'user3', 'user4', 'user5', 'user5', 'user5', 'user5', 'user5', 'user6', 'user7'],
    'tweet': ['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17'],
    'Label': ['dem', 'dem', 'neu', 'rep', 'rep', 'dem', 'neu', 'rep', 'neu', 'dem', 'dem', 'dem', 'dem', 'rep', 'neu', 'neu', 'rep']
})

V_ejemplo = np.array([-0.8, 0.9, -0.1, -0.9, 0.0, 0.0, 0.0])
users_ejemplo = np.array(['user1', 'user2', 'user3', 'user4', 'user5', 'user6', 'user7'])

# 2. Calcular entropías h_i (Función auxiliar)
entropies_ejemplo = calcular_entropia_usuario(data_tweets_ejemplo)
print("Entropías calculadas (h_i):")
print(entropies_ejemplo)

# 3. Calcular Índice Beta
beta_ejemplo = 0.5
print(f"\n--- Cálculo Índice Beta con beta={beta_ejemplo} ---")
print(f"Vector V: {V_ejemplo}")
print(f"Usuarios: {users_ejemplo}")
indice_beta_calculado = calcular_IB(V_ejemplo, beta_ejemplo, entropies_ejemplo, users_ejemplo)
print(f"Índice Beta calculado: {indice_beta_calculado}")


In [ ]:
# Aplicación a nuestras muestras

IB_16 = calcular_IB(V_16, 0, calcular_entropia_usuario(data_table_16), V_df_16.index)
IB_20 = calcular_IB(V_20, 0, calcular_entropia_usuario(data_table_20), V_df_20.index)
IB_24 = calcular_IB(V_24, 0, calcular_entropia_usuario(data_table_24), V_df_24.index)

print(IB_16)
print(IB_20)
print(IB_24)